**LIBRARY**

In [35]:
!python --version

Python 3.12.4


In [36]:
!apt-get install python3.12

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
python3.12 is already the newest version (3.12.4-1+jammy1).
The following packages were automatically installed and are no longer required:
  blt libaio1 libboost-atomic-dev libboost-atomic1.74-dev libboost-atomic1.74.0 libboost-chrono-dev
  libboost-chrono1.74-dev libboost-chrono1.74.0 libboost-container-dev libboost-container1.74-dev
  libboost-container1.74.0 libboost-context-dev libboost-context1.74-dev libboost-context1.74.0
  libboost-coroutine-dev libboost-coroutine1.74-dev libboost-coroutine1.74.0 libboost-date-time-dev
  libboost-date-time1.74-dev libboost-date-time1.74.0 libboost-exception-dev
  libboost-exception1.74-dev libboost-fiber-dev libboost-fiber1.74-dev libboost-fiber1.74.0
  libboost-filesystem-dev libboost-filesystem1.74-dev libboost-filesystem1.74.0 libboost-graph-dev
  libboost-graph-parallel-dev libboost-graph-parallel1.74-dev libboost-graph-parallel1.74.0
  libboos

In [37]:
!python3.12 --version

Python 3.12.4


In [38]:
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.10 1
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.12 2

In [39]:
!python --version

Python 3.12.4


In [40]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
import plotly.graph_objects as go
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, LSTM, Conv1D, MaxPooling1D, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

**LOAD DATASET**

In [41]:
bbca = pd.read_csv('BBCA_clean_modified.csv')
bbni = pd.read_csv('BBNI_clean_modified.csv')
bbri = pd.read_csv('BBRI_clean_modified.csv')
bmri = pd.read_csv('BMRI_clean_modified.csv')

In [42]:
def preprocess_data(df):
    df = df.dropna().reset_index(drop=True)
    features = df[['Open', 'High', 'Low', 'Volume', 'H-L', 'O-C', '7 DAYS MA', '14 DAYS MA', '21 DAYS MA', '7 DAYS STD DEV']]
    target = df['Close']
    scaler = MinMaxScaler()
    features = scaler.fit_transform(features)
    return train_test_split(features, target, test_size=0.2, random_state=42), df['Date']

In [43]:
def lstm_model(input_shape):
    model = Sequential()
    model.add(LSTM(128, return_sequences=True, input_shape=input_shape))
    model.add(Dropout(0.3))
    model.add(LSTM(64, return_sequences=False))
    model.add(Dropout(0.3))
    model.add(Dense(32))
    model.add(Dense(1))
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')
    return model

def cnn_model(input_shape):
    model = Sequential()
    model.add(Conv1D(filters=128, kernel_size=3, activation='relu', input_shape=input_shape))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(0.3))
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(0.3))
    model.add(Flatten())
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')
    return model

In [44]:
def hyperparameter_tuning(model_builder, X_train, y_train, X_val, y_val):
    best_params = None
    best_score = float('inf')
    param_grid = {
        'batch_size': [16, 32],
        'epochs': [50, 100]
    }
    for batch_size in param_grid['batch_size']:
        for epochs in param_grid['epochs']:
            model = model_builder((X_train.shape[1], X_train.shape[2]))
            history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_val, y_val),
                                callbacks=[EarlyStopping(patience=10)], verbose=0)
            score = np.min(history.history['val_loss'])
            if score < best_score:
                best_score = score
                best_params = {'batch_size': batch_size, 'epochs': epochs}
    print(f"Best: {best_score} using {best_params}")
    return best_params

# Evaluate Model with Cross-Validation
def evaluate_model_with_cv(model_builder, X_train, X_test, y_train, y_test, model_name):
    best_params = hyperparameter_tuning(model_builder, X_train, y_train, X_test, y_test)
    model = model_builder((X_train.shape[1], X_train.shape[2]))
    checkpoint = ModelCheckpoint(f'{model_name}.h5', monitor='val_loss', save_best_only=True, verbose=0)
    history = model.fit(X_train, y_train, epochs=best_params['epochs'], batch_size=best_params['batch_size'], validation_data=(X_test, y_test),
                        callbacks=[EarlyStopping(patience=10), checkpoint], verbose=1)
    model = load_model(f'{model_name}.h5')

    train_predictions = model.predict(X_train)
    test_predictions = model.predict(X_test)

    train_rmse = np.sqrt(mean_squared_error(y_train, train_predictions))
    test_rmse = np.sqrt(mean_squared_error(y_test, test_predictions))
    train_mae = mean_absolute_error(y_train, train_predictions)
    test_mae = mean_absolute_error(y_test, test_predictions)

    return train_rmse, test_rmse, train_mae, test_mae, train_predictions, test_predictions

In [45]:
def visualize_model_performance(dates, y_train, y_test, y_train_pred, y_test_pred, title):
    train_dates = dates[:len(y_train)]
    test_dates = dates[len(y_train):len(y_train) + len(y_test)]
    all_dates = dates[:len(y_train) + len(y_test)]

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=train_dates, y=y_train, mode='markers', name='Train Actual', marker=dict(color='gray')))
    fig.add_trace(go.Scatter(x=test_dates, y=y_test, mode='markers', name='Test Actual', marker=dict(color='red')))

    # Combine train and test predictions for a single prediction line
    all_predictions = np.concatenate((y_train_pred, y_test_pred))

    fig.add_trace(go.Scatter(x=all_dates, y=all_predictions.flatten(), mode='lines', name='Predictions', line=dict(color='orange', width=2, dash='dot')))

    fig.update_layout(
        title=title,
        xaxis=dict(
            rangeslider=dict(visible=True),
            type='date'
        ),
        yaxis=dict(title='Close Price IDR'),
        xaxis_range=[all_dates.iloc[0], all_dates.iloc[-1]]
    )
    fig.show()

In [46]:
def process_dataset(df, dataset_name):
    (X_train, X_test, y_train, y_test), dates = preprocess_data(df)
    X_train_dl = np.expand_dims(X_train, axis=2)
    X_test_dl = np.expand_dims(X_test, axis=2)

    # LSTM Model
    lstm_train_rmse, lstm_test_rmse, lstm_train_mae, lstm_test_mae, lstm_train_predictions, lstm_test_predictions = evaluate_model_with_cv(lstm_model, X_train_dl, X_test_dl, y_train, y_test, f'{dataset_name}_LSTM')

    # CNN Model
    cnn_train_rmse, cnn_test_rmse, cnn_train_mae, cnn_test_mae, cnn_train_predictions, cnn_test_predictions = evaluate_model_with_cv(cnn_model, X_train_dl, X_test_dl, y_train, y_test, f'{dataset_name}_CNN')

    # Visualize LSTM and CNN Performance
    visualize_model_performance(dates, y_train, y_test, lstm_train_predictions, lstm_test_predictions, f'Enhanced LSTM Model for {dataset_name}')
    visualize_model_performance(dates, y_train, y_test, cnn_train_predictions, cnn_test_predictions, f'Enhanced CNN Model for {dataset_name}')

    return {
        'Dataset': dataset_name,
        'LSTM_Train_RMSE': lstm_train_rmse,
        'LSTM_Test_RMSE': lstm_test_rmse,
        'LSTM_Train_MAE': lstm_train_mae,
        'LSTM_Test_MAE': lstm_test_mae,
        'CNN_Train_RMSE': cnn_train_rmse,
        'CNN_Test_RMSE': cnn_test_rmse,
        'CNN_Train_MAE': cnn_train_mae,
        'CNN_Test_MAE': cnn_test_mae
    }

In [47]:
results=[]

In [48]:
BBCARES=process_dataset(bbca, 'BBCA')

Best: 10743.6943359375 using {'batch_size': 16, 'epochs': 100}
Epoch 1/100
62/62 [==============================] - 8s 57ms/step - loss: 16633966.0000 - val_loss: 16760424.0000
Epoch 2/100
 8/62 [==>...........................] - ETA: 0s - loss: 16093349.0000

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning:

You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.



62/62 [==============================] - 1s 20ms/step - loss: 16121267.0000 - val_loss: 16148033.0000
Epoch 3/100
62/62 [==============================] - 1s 19ms/step - loss: 15391374.0000 - val_loss: 15291863.0000
Epoch 4/100
62/62 [==============================] - 1s 20ms/step - loss: 14428618.0000 - val_loss: 14206179.0000
Epoch 5/100
62/62 [==============================] - 1s 20ms/step - loss: 13251495.0000 - val_loss: 12939407.0000
Epoch 6/100
62/62 [==============================] - 2s 31ms/step - loss: 11941993.0000 - val_loss: 11550253.0000
Epoch 7/100
62/62 [==============================] - 2s 29ms/step - loss: 10511684.0000 - val_loss: 10097809.0000
Epoch 8/100
62/62 [==============================] - 1s 19ms/step - loss: 9064913.0000 - val_loss: 8654068.0000
Epoch 9/100
62/62 [==============================] - 1s 19ms/step - loss: 7645484.5000 - val_loss: 7261042.5000
Epoch 10/100
62/62 [==============================] - 1s 19ms/step - loss: 6392728.5000 - val_loss: 5986

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning:

You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.



62/62 [==============================] - 0s 4ms/step - loss: 14187878.0000 - val_loss: 9962305.0000
Epoch 3/100
62/62 [==============================] - 0s 4ms/step - loss: 3703772.7500 - val_loss: 204424.7500
Epoch 4/100
62/62 [==============================] - 0s 5ms/step - loss: 379849.0938 - val_loss: 201551.9375
Epoch 5/100
62/62 [==============================] - 0s 4ms/step - loss: 348270.3125 - val_loss: 181530.5000
Epoch 6/100
62/62 [==============================] - 0s 4ms/step - loss: 355590.3438 - val_loss: 178775.0156
Epoch 7/100
62/62 [==============================] - 0s 5ms/step - loss: 366973.5312 - val_loss: 176103.0312
Epoch 8/100
62/62 [==============================] - 0s 5ms/step - loss: 383830.6562 - val_loss: 153826.6719
Epoch 9/100
62/62 [==============================] - 0s 4ms/step - loss: 347104.1875 - val_loss: 164930.0156
Epoch 10/100
62/62 [==============================] - 0s 4ms/step - loss: 336005.6562 - val_loss: 156778.3281
Epoch 11/100
62/62 [======

In [49]:
results.append(BBCARES)
pd.DataFrame(BBCARES, index=[0]).T

,0
Dataset,BBCA
LSTM_Train_RMSE,73.653286
LSTM_Test_RMSE,96.326771
LSTM_Train_MAE,48.723934
LSTM_Test_MAE,56.328201
CNN_Train_RMSE,221.397594
CNN_Test_RMSE,202.839567
CNN_Train_MAE,123.597662
CNN_Test_MAE,130.808301


In [50]:
BBNIRES=process_dataset(bbni, 'BBNI')

Best: 4414.02099609375 using {'batch_size': 16, 'epochs': 100}
Epoch 1/100
62/62 [==============================] - 6s 40ms/step - loss: 17854896.0000 - val_loss: 17034988.0000
Epoch 2/100
 4/62 [>.............................] - ETA: 1s - loss: 16360222.0000

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning:

You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.



62/62 [==============================] - 2s 26ms/step - loss: 17326604.0000 - val_loss: 16404815.0000
Epoch 3/100
62/62 [==============================] - 1s 19ms/step - loss: 16550762.0000 - val_loss: 15516879.0000
Epoch 4/100
62/62 [==============================] - 1s 18ms/step - loss: 15537265.0000 - val_loss: 14395905.0000
Epoch 5/100
62/62 [==============================] - 1s 16ms/step - loss: 14282030.0000 - val_loss: 13084797.0000
Epoch 6/100
62/62 [==============================] - 1s 17ms/step - loss: 12861980.0000 - val_loss: 11631558.0000
Epoch 7/100
62/62 [==============================] - 1s 16ms/step - loss: 11357782.0000 - val_loss: 10110088.0000
Epoch 8/100
62/62 [==============================] - 1s 17ms/step - loss: 9769139.0000 - val_loss: 8586711.0000
Epoch 9/100
62/62 [==============================] - 1s 16ms/step - loss: 8234914.0000 - val_loss: 7114137.5000
Epoch 10/100
62/62 [==============================] - 1s 17ms/step - loss: 6768757.0000 - val_loss: 5767

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning:

You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.



31/31 [==============================] - 0s 6ms/step - loss: 17768592.0000 - val_loss: 16789302.0000
Epoch 3/100
31/31 [==============================] - 0s 6ms/step - loss: 16255687.0000 - val_loss: 13988778.0000
Epoch 4/100
31/31 [==============================] - 0s 7ms/step - loss: 11438648.0000 - val_loss: 7242187.0000
Epoch 5/100
31/31 [==============================] - 0s 6ms/step - loss: 3806231.7500 - val_loss: 720137.3750
Epoch 6/100
31/31 [==============================] - 0s 6ms/step - loss: 414537.3438 - val_loss: 57407.2539
Epoch 7/100
31/31 [==============================] - 0s 5ms/step - loss: 287015.6875 - val_loss: 60305.9609
Epoch 8/100
31/31 [==============================] - 0s 6ms/step - loss: 282555.1562 - val_loss: 53685.9023
Epoch 9/100
31/31 [==============================] - 0s 7ms/step - loss: 269886.7188 - val_loss: 54457.2070
Epoch 10/100
31/31 [==============================] - 0s 5ms/step - loss: 309625.2812 - val_loss: 57587.7344
Epoch 11/100
31/31 [===

In [51]:
results.append(BBNIRES)
pd.DataFrame(BBNIRES, index=[0]).T

,0
Dataset,BBNI
LSTM_Train_RMSE,86.922767
LSTM_Test_RMSE,84.589249
LSTM_Train_MAE,64.517777
LSTM_Test_MAE,64.571777
CNN_Train_RMSE,221.242184
CNN_Test_RMSE,187.317113
CNN_Train_MAE,124.88188
CNN_Test_MAE,122.139267


In [52]:
BBRIRES=process_dataset(bbri, 'BBRI')

Best: 12469.482421875 using {'batch_size': 16, 'epochs': 100}
Epoch 1/100
62/62 [==============================] - 8s 53ms/step - loss: 14995389.0000 - val_loss: 14680873.0000
Epoch 2/100
 5/62 [=>............................] - ETA: 1s - loss: 14578472.0000

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning:

You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.



62/62 [==============================] - 1s 18ms/step - loss: 14523625.0000 - val_loss: 14115323.0000
Epoch 3/100
62/62 [==============================] - 1s 17ms/step - loss: 13836732.0000 - val_loss: 13316052.0000
Epoch 4/100
62/62 [==============================] - 1s 17ms/step - loss: 12921931.0000 - val_loss: 12307383.0000
Epoch 5/100
62/62 [==============================] - 1s 16ms/step - loss: 11812403.0000 - val_loss: 11133109.0000
Epoch 6/100
62/62 [==============================] - 1s 17ms/step - loss: 10597050.0000 - val_loss: 9854897.0000
Epoch 7/100
62/62 [==============================] - 1s 16ms/step - loss: 9233027.0000 - val_loss: 8518752.0000
Epoch 8/100
62/62 [==============================] - 1s 16ms/step - loss: 7904111.5000 - val_loss: 7197991.0000
Epoch 9/100
62/62 [==============================] - 1s 17ms/step - loss: 6581148.0000 - val_loss: 5951444.0000
Epoch 10/100
62/62 [==============================] - 1s 17ms/step - loss: 5396567.5000 - val_loss: 4820592

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning:

You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.



62/62 [==============================] - 0s 4ms/step - loss: 12419919.0000 - val_loss: 7950542.5000
Epoch 3/100
62/62 [==============================] - 0s 4ms/step - loss: 2759116.5000 - val_loss: 185162.4375
Epoch 4/100
62/62 [==============================] - 0s 4ms/step - loss: 318231.5938 - val_loss: 195643.5625
Epoch 5/100
62/62 [==============================] - 0s 5ms/step - loss: 336438.5625 - val_loss: 176509.2969
Epoch 6/100
62/62 [==============================] - 0s 4ms/step - loss: 306932.7500 - val_loss: 180738.5469
Epoch 7/100
62/62 [==============================] - 0s 4ms/step - loss: 342706.4062 - val_loss: 174747.2500
Epoch 8/100
62/62 [==============================] - 0s 4ms/step - loss: 318983.5938 - val_loss: 167751.8281
Epoch 9/100
62/62 [==============================] - 0s 4ms/step - loss: 284607.5000 - val_loss: 152565.4844
Epoch 10/100
62/62 [==============================] - 0s 5ms/step - loss: 327392.1875 - val_loss: 149337.5312
Epoch 11/100
62/62 [======

In [53]:
results.append(BBRIRES)
pd.DataFrame(BBRIRES, index=[0]).T

,0
Dataset,BBRI
LSTM_Train_RMSE,88.285253
LSTM_Test_RMSE,104.192619
LSTM_Train_MAE,58.573008
LSTM_Test_MAE,64.26303
CNN_Train_RMSE,196.458486
CNN_Test_RMSE,210.18538
CNN_Train_MAE,113.744837
CNN_Test_MAE,119.686358


In [54]:
BMRIRES=process_dataset(bmri, 'BMRI')

Best: 12367.8408203125 using {'batch_size': 16, 'epochs': 100}
Epoch 1/100
62/62 [==============================] - 6s 31ms/step - loss: 53306748.0000 - val_loss: 51636532.0000
Epoch 2/100
 9/62 [===>..........................] - ETA: 0s - loss: 54007964.0000

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning:

You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.



62/62 [==============================] - 1s 17ms/step - loss: 52359620.0000 - val_loss: 50512532.0000
Epoch 3/100
62/62 [==============================] - 1s 19ms/step - loss: 50984344.0000 - val_loss: 48910652.0000
Epoch 4/100
62/62 [==============================] - 2s 26ms/step - loss: 49132872.0000 - val_loss: 46843464.0000
Epoch 5/100
62/62 [==============================] - 2s 26ms/step - loss: 46873956.0000 - val_loss: 44364608.0000
Epoch 6/100
62/62 [==============================] - 1s 17ms/step - loss: 44120216.0000 - val_loss: 41509452.0000
Epoch 7/100
62/62 [==============================] - 1s 17ms/step - loss: 41170352.0000 - val_loss: 38369284.0000
Epoch 8/100
62/62 [==============================] - 1s 17ms/step - loss: 37877560.0000 - val_loss: 35030912.0000
Epoch 9/100
62/62 [==============================] - 1s 17ms/step - loss: 34423840.0000 - val_loss: 31571680.0000
Epoch 10/100
62/62 [==============================] - 1s 17ms/step - loss: 30890308.0000 - val_loss:

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning:

You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.



62/62 [==============================] - 0s 6ms/step - loss: 48103024.0000 - val_loss: 37038460.0000
Epoch 3/50
62/62 [==============================] - 0s 6ms/step - loss: 18737766.0000 - val_loss: 1819906.0000
Epoch 4/50
62/62 [==============================] - 0s 6ms/step - loss: 864969.2500 - val_loss: 155952.9062
Epoch 5/50
62/62 [==============================] - 0s 7ms/step - loss: 729514.6875 - val_loss: 183586.6094
Epoch 6/50
62/62 [==============================] - 0s 6ms/step - loss: 767764.1250 - val_loss: 138878.7969
Epoch 7/50
62/62 [==============================] - 0s 7ms/step - loss: 728521.3125 - val_loss: 162326.0000
Epoch 8/50
62/62 [==============================] - 0s 7ms/step - loss: 694075.7500 - val_loss: 134276.2812
Epoch 9/50
62/62 [==============================] - 0s 7ms/step - loss: 766247.5625 - val_loss: 143071.6250
Epoch 10/50
62/62 [==============================] - 0s 4ms/step - loss: 675421.6250 - val_loss: 117461.5312
Epoch 11/50
62/62 [============

In [55]:
results.append(BMRIRES)
pd.DataFrame(BMRIRES, index=[0]).T

,0
Dataset,BMRI
LSTM_Train_RMSE,139.700307
LSTM_Test_RMSE,124.543387
LSTM_Train_MAE,96.49377
LSTM_Test_MAE,89.957637
CNN_Train_RMSE,236.613164
CNN_Test_RMSE,199.823135
CNN_Train_MAE,126.053657
CNN_Test_MAE,121.240441


In [56]:
resultsall=pd.DataFrame(results).T
resultsall

,0,1,2,3
Dataset,BBCA,BBNI,BBRI,BMRI
LSTM_Train_RMSE,73.653286,86.922767,88.285253,139.700307
LSTM_Test_RMSE,96.326771,84.589249,104.192619,124.543387
LSTM_Train_MAE,48.723934,64.517777,58.573008,96.49377
LSTM_Test_MAE,56.328201,64.571777,64.26303,89.957637
CNN_Train_RMSE,221.397594,221.242184,196.458486,236.613164
CNN_Test_RMSE,202.839567,187.317113,210.18538,199.823135
CNN_Train_MAE,123.597662,124.88188,113.744837,126.053657
CNN_Test_MAE,130.808301,122.139267,119.686358,121.240441
